In [21]:
import numpy as np
import matplotlib.pyplot as plt
from ReadData import getCVRPFileData

In [22]:
def CVRP_function(routes):
    distance = 0
    for i in np.arange(len(routes)-1):
        distance += np.sqrt((routes[i][0] - routes[i+1][0])**2 + (routes[i][1] - routes[i+1][1])**2)
    return distance

def getAllDistances(data, routes):
    allDistances = 0
    for route in routes:
        coordenates = []
        coordenates.append(data[0][1])
        for node in route:
            coordenates.append(data[node][1])
        coordenates.append(data[0][1])
        allDistances += CVRP_function(coordenates)
    return allDistances

In [23]:
import numpy as np
import sys
import math
import time
import random
from typing import Callable
from itertools import chain

ListOfListType = list[list[int or float]]
ObjetiveType = [[ListOfListType, ListOfListType], int or float]

class AntColony(object):

    def __init__(self, number_ants = 5, data = [], capacity = 0, iterations = 20, ro = 0.5, alpha = 1, betha = 1):
        self.number_ants = number_ants
        self.data = data
        self.capacity = capacity
        self.iterations = iterations
        self.ro = ro
        self.alpha = alpha
        self.betha = betha

    def create_nodes(self) -> list:
        first_solution = list(range(1, (len(self.data))))
        random.shuffle(first_solution)
        return first_solution

    def getDemand(self, route: list) -> int or float:
        demand = 0
        for node in route:
            demand += self.data[node][2]
        return demand
    
    def is_under_demand(self, actual_solution: list) -> bool:
        demand = self.getDemand(actual_solution)
    
        is_under_demand_ = True
        if(demand > self.capacity):
            is_under_demand_ = False
                    
        return is_under_demand_

    def create_dictionary(self, size: int, value: int) -> dict:
        dictionary = dict()
        for i in np.arange(size):
            for j in np.arange(size):
                if(i < j):
                    key = str(i+1) + str(j+1)
                    dictionary.setdefault(key, value)
        return dictionary

    def init_pheromons(self, solution: list, value: int) -> dict:
        pheromons = self.create_dictionary(size=len(solution), value=value)
        return pheromons
    
    def roulette_method(self, costs: list):
        lista_sin_negativos = [elemento for elemento in costs if elemento != -1]
        lista_inversa = [1 / x for x in lista_sin_negativos]

        suma = sum(lista_inversa)
        lista_normalizada = [elemento / suma for elemento in lista_inversa]

        ruleta = []
        acumulado = 0
        for peso in lista_normalizada:
            acumulado += peso
            ruleta.append(acumulado)

        numero_aleatorio = random.random()
        indice_ganador = None
        for i, probabilidad in enumerate(ruleta):
            if numero_aleatorio <= probabilidad:
                indice_ganador = i
                break

        return costs.index(lista_sin_negativos[indice_ganador])

    def calculate_costs(self, nodes: list, actual_solution: list, actual_route: list) ->list:
        costs = []

        for node in nodes:
            actual_solution_copy = actual_solution.copy()
            actual_route_copy = actual_route.copy()
            actual_route_copy.append(node)
            
            if(self.is_under_demand(actual_route_copy)):
                actual_solution_copy.append(actual_route_copy)
                costs.append(self.cost_function(self.data, actual_solution_copy))
            else:
                costs.append(0)
        return costs

    def calculate_probabilitys(self, nodes: list, actual_route: list, pheromons: dict, costs: list):
        actual_node = actual_route[len(actual_route) - 1]
        probabilitys = []
        divisor = 0

        for i,node in enumerate(nodes):
            if(costs[i]):
                if(actual_node < node):
                    a1, a2 = actual_node, node
                else:
                    a1, a2 = node, actual_node
                
                key = str(a1) + str(a2)
                pherom = pheromons[key]
                divisor += (math.pow(pherom, self.alpha)) * (math.pow((1 / costs[i]), self.betha))
        
        if(not divisor):
           return None

        for i,node in enumerate(nodes):
            if(costs[i]):
                if(actual_node < node):
                    a1, a2 = actual_node, node
                else:
                    a1, a2 = node, actual_node
                
                key = str(a1) + str(a2)
                pherom = pheromons[key]
    
                probabilitys.append((math.pow(pherom, self.alpha) * math.pow(((1 / costs[i]) / divisor), self.betha)))
            else:
                probabilitys.append(-1)
        
        return probabilitys

    def build_solution(self, solution: list, pheromons: dict):
        actual_solution = []
        index = 0
        nodes = solution.copy()
        sum_pheromons = 0
        
        initial_node = random.choice(nodes)
        actual_route = []
        actual_route.append(initial_node)
        
        nodes.remove(initial_node)

        while(len(nodes) > 0):
            costs = self.calculate_costs(nodes=nodes, actual_solution=actual_solution, actual_route=actual_route)
            probabilitys = self.calculate_probabilitys(nodes=nodes, actual_route=actual_route, pheromons=pheromons, costs=costs)
            if(probabilitys):
                index_winner = self.roulette_method(costs=probabilitys)
                sum_pheromons += costs[index_winner]
                actual_route.append(nodes[index_winner])
                nodes.remove(nodes[index_winner])
            else:
                actual_solution.append(actual_route)
                initial_node = random.choice(nodes)
                actual_route = []
                actual_route.append(initial_node)
                nodes.remove(initial_node)
        actual_solution.append(actual_route)

        return [self.cost_function(self.data, actual_solution), actual_solution, sum_pheromons]

    def get_best_solution(self, solutions: ListOfListType) -> ListOfListType:
        sort_solutions = sorted(solutions, key=lambda x: x[0], reverse=False)

        return sort_solutions[0][0], sort_solutions[0][1]
    
    def evaporate_pheromons(self,pheromons: dict, ro:float, size: int):
        for i in np.arange(size):
            for j in np.arange(size):
                if(i < j):
                    key = str(i+1) + str(j+1)
                    pheromons[key] = pheromons[key] * (1 - ro)
                    

    def update_pheromons(self, results: list, pheromons:dict, actual_solution: list, ro: float):
        self.evaporate_pheromons(pheromons=pheromons, ro=ro, size=len(actual_solution))

        for result in results:
            solution = result[1]
            solution = list(chain(*solution))
            sum_pheromons = (1 / result[2])
            for i in np.arange(len(solution) - 1):
                if(solution[i] < solution[i+1]):
                    a1, a2 = solution[i], solution[i+1]
                else:
                    a1, a2 = solution[i+1], solution[i]
                key = str(a1) + str(a2)
                pheromons[key] = pheromons[key] + sum_pheromons
        return pheromons
                

    def fit(self, objetive: ObjetiveType):
        self.cost_function = objetive
        self.best_cost = float("inf")
        self.costs = []
        inicio = time.time()
        best_solution = []
        best_cost = float("inf")
        nodes = self.create_nodes()
        pheromons = self.init_pheromons(solution=nodes, value=1)

        i = 0
        while(i < self.iterations):
            solutions = []
            for _ in np.arange(self.number_ants):
                solutions.append(self.build_solution(solution=nodes, pheromons=pheromons))
            
            best_cost, best_solution = self.get_best_solution(solutions=solutions)

            if(best_cost < self.best_cost):
                self.best_cost = best_cost
                self.best_solution = best_solution
            self.costs.append(self.best_cost)
            
            pheromons = self.update_pheromons(results=solutions, actual_solution=nodes, pheromons=pheromons, ro=self.ro)
            
            fin = time.time()
            total_time = fin - inicio
            sys.stderr.write('\r%d Iteration: | Cost %f | Time: %f' % (i, self.best_cost, total_time))
            time.sleep(0)  
            sys.stderr.flush()
            i += 1
        self.best_cost = best_cost
        self.best_solution = best_solution

In [24]:
data, capacity = getCVRPFileData('./data/A-n15-k5.txt')

ac = AntColony(data=data, capacity=capacity, iterations=5000,  number_ants=1, ro=0.5, alpha=1, betha=2)

ac.fit(getAllDistances)


4999 Iteration: | Cost 461.318926 | Time: 9.917467

In [25]:
print(ac.best_solution)

[[7, 3], [1, 12], [9, 13, 10, 15], [14, 11], [6], [8], [4, 5], [2]]
